In [12]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("AZURE_OPENAI_API_KEY")
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
client = AzureOpenAI(
    api_key=api_key,
    api_version="2024-02-01",
    azure_endpoint=endpoint,
)
def call_llm(client, text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": text,
            },
        ]
    )
    return response.choices[0].message.content


In [15]:
prompt = """以下のユーザーの要求から埋めるべきslotをJSON形式で抽出してください
日付はmm/dd形式で出力してください。今日は{{today}}です。

また、ユーザーからのリクエストに対してFAQのAnswerリストに関連する場合は返信を行います。
関連するものがある場合はanswerにその内容を記載してください。ない場合は空欄にしてください。
Answerリスト:
- ランチの営業時間は11:00から15:00
- ディナーの営業時間は17:00から23:00
- 駐車場は2台停められます。
- ランチは席代がかかりませんが、ディナーは席代がかかります。
- ランチは予約できません。

### slot

- 名前
- 日付
- 人数
- 時間
- answer

### output_format
{{"名前": "", "日付": "", "人数": "", "時間": "", "answer": ""}}

### ユーザーの要求
{{user_request}}

### 予測結果"""

formatted_prompt = prompt.format(today="2023/10/09", user_request="ランチの予約をしたいです。")
print(formatted_prompt)

以下のユーザーの要求から埋めるべきslotをJSON形式で抽出してください
日付はmm/dd形式で出力してください。今日は{today}です。

また、ユーザーからのリクエストに対してFAQのAnswerリストに関連する場合は返信を行います。
関連するものがある場合はanswerにその内容を記載してください。ない場合は空欄にしてください。
Answerリスト:
- ランチの営業時間は11:00から15:00
- ディナーの営業時間は17:00から23:00
- 駐車場は2台停められます。
- ランチは席代がかかりませんが、ディナーは席代がかかります。
- ランチは予約できません。

### slot

- 名前
- 日付
- 人数
- 時間
- answer

### output_format
{"名前": "", "日付": "", "人数": "", "時間": "", "answer": ""}

### ユーザーの要求
{user_request}

### 予測結果


In [30]:
import datetime

# 現在の日付を取得し、'YYYY/MM/DD'形式にフォーマット
today = datetime.datetime.now().strftime('%Y/%m/%d')

# プロンプトの定義
prompt_template = """以下のユーザーの要求から、指定された情報をJSON形式で抽出してください。

- **日付**は`mm/dd`形式で出力してください。今日は{today}です。
- ユーザーのリクエストにFAQの回答リストに関連する内容がある場合、"answer"フィールドにその内容を記載してください。関連がない場合は空欄にしてください。

**FAQの回答リスト:**
- ランチの営業時間は11:00から15:00です。
- ディナーの営業時間は17:00から23:00です。
- 駐車場は2台分ございます。
- ランチは席代がかかりませんが、ディナーは席代がかかります。
- ランチは予約を受け付けておりません。

### 必要な情報

- 名前
- 日付
- 人数
- 時間
- answer

### 出力フォーマット
{{"名前": "", "日付": "", "人数": "", "時間": "", "answer": ""}}

ユーザーの要求
{user_request}

予測結果
"""

# ユーザーの要求の例
user_request = "明日4人で予約したいんですけど、ランチって何時からでしたっけ"

# フォーマット
llm_prompt = prompt_template.format(today=today, user_request=user_request)

# 結果表示
print(llm_prompt)

以下のユーザーの要求から、指定された情報をJSON形式で抽出してください。

- **日付**は`mm/dd`形式で出力してください。今日は2024/10/17です。
- ユーザーのリクエストにFAQの回答リストに関連する内容がある場合、"answer"フィールドにその内容を記載してください。関連がない場合は空欄にしてください。

**FAQの回答リスト:**
- ランチの営業時間は11:00から15:00です。
- ディナーの営業時間は17:00から23:00です。
- 駐車場は2台分ございます。
- ランチは席代がかかりませんが、ディナーは席代がかかります。
- ランチは予約を受け付けておりません。

### 必要な情報

- 名前
- 日付
- 人数
- 時間
- answer

### 出力フォーマット
{"名前": "", "日付": "", "人数": "", "時間": "", "answer": ""}

ユーザーの要求
明日4人で予約したいんですけど、ランチって何時からでしたっけ

予測結果



In [32]:
import datetime
today = datetime.datetime.now().strftime('%Y/%m/%d')
print(today)
user_request = "明日4人で予約したいんですけど、ランチって何時からでしたっけ"

llm_prompt = prompt_template.format(today=today, user_request=user_request)
call_llm(client, llm_prompt)


2024/10/17


'```json\n{\n  "名前": "",\n  "日付": "10/18",\n  "人数": "4",\n  "時間": "",\n  "answer": "ランチの営業時間は11:00から15:00です。"\n}\n```'

In [ ]:
import os
from openai import AzureOpenAI
from src.utils import get_custom_logger

from dotenv import load_dotenv
import queue

load_dotenv()

logger = get_custom_logger()

system_prompt = """
あなたは飲食店の店員です。
ユーザーからのメッセージに対してFAQのAnswerリストに関連する場合は返信を行います。
関連するものがない場合は、空文字を返してください。
Answerリスト:
- ランチの営業時間は11:00から15:00
- ディナーの営業時間は17:00から23:00
- 駐車場は2台停められます。
- ランチは席代がかかりませんが、ディナーは席代がかかります。
- ランチは予約できません。
"""

class LLMBridge:
    def __init__(self):
        self.input_queue = queue.Queue()
        self.output_queue = queue.Queue()
        self.stream_sid = None

        api_key = os.environ.get("AZURE_OPENAI_API_KEY")
        endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
        self.client = AzureOpenAI(
            api_key=api_key,
            api_version="2024-02-01",
            azure_endpoint=endpoint,
        )

    def add_request(self, text):
        if text:
            logger.info(f"Add request: {text}")
            self.input_queue.put(text)

    def response_loop(self):
        logger.info("Response loop called.")
        while True:
            text = self.input_queue.get()
            if text is None:
                logger.info("Received exit signal.")
                break
            else:
                response = self.call_llm(text)
                self.output_queue.put(response)
        logger.info("Response loop ended.")

    def get_response(self, timeout=None):
        try:
            response = self.output_queue.get(timeout=timeout)
            return response
        except queue.Empty:
            logger.info("Response queue is empty.")
            return None

    def terminate(self):
        self.input_queue.put(None)

    def call_llm(self, text):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": text,
                },
            ]
        )
        return response.choices[0].message.content


import threading
import time

llm_bridge = LLMBridge()
t_llm = threading.Thread(target=llm_bridge.response_loop)
t_llm.start()
llm_bridge.add_request("ランチの営業時間は何時からですか？")

tic = time.time()
response = llm_bridge.get_response(timeout=10)  # 最大10秒待機
if response:
    logger.info(f"Response: {response}")
else:
    logger.info("No response received.")

toc = time.time() - tic
logger.info(f"Elapsed time: {toc}")

llm_bridge.add_request("ディナーの営業時間は何時からですか？")
tic = time.time()
response = llm_bridge.get_response()
logger.info(f"Response: {response}")
toc = time.time() - tic
logger.info(f"Elapsed time: {toc}")

llm_bridge.terminate()
t_llm.join()  # スレッドの終了を待つ